# Knowledge

## Biology

## Math

### Gausian random field

A true Gaussian random field: generate a height map where each value is random, but related to neighbors → correlated spatial noise.

### Sum of randomly places gausian contributers

This is what we are doing now to simulate our field:

1. Place gausian sources randomly:

``` python
repellent_positions = [
    (np.random.uniform(0.0, 0.45) if np.random.rand() < 0.5 else np.random.uniform(0.55, 1.0), 
     np.random.uniform(0.0, 0.2)) 
    for _ in range(num_repellents)
]
```

2. Calculate the sum of gausian points to create a field:

``` python
repellent_field = np.zeros_like(x)
for rx, ry in repellent_positions:
    repellent_field += repellent_strength * np.exp(-((x - rx)**2 + (y - ry)**2) / (2 * sigma_repellent**2))
```
### Gradient descent

Considers the steepest change in intensity and takes a step in that direction.

__NB sensitive to step size__

Let’s say an attractor is at (0.5, 0.2) and the neuron is at (0.5, 0.25).

With __small step_size__ = 0.005, the axon moves gradually, allowing a smooth descent toward the attractor.

With __large step_size__ = 0.05, it might go too far — say to (0.5, 0.15) — skipping the attractor’s sweet spot, and might even end up exiting the field prematurely.

## Code:

### np.gradient()

Used to compute x,y gradient:

``` python
grad_y, grad_x = np.gradient(noisy_field)
```

__This tells the axon how to navigate the generated field.__ 

Differences that can be used:

1. Central - what np.gradient is using (except for edge cases)
edge cases:
2. Forward - compares the current point to the future
3. Backward - compares the current to the previous 

__NB np.gradient uses only the neighbours along the x and y - partial derivatives not all 8 neighbours__

uses:

•   ↑   •

←   ⊗   →

•   ↓   •

not:

↖ ↑ ↗

← ⊗ →

↙ ↓ ↘

---

#### Central Differences

For **most** points (not at the edges), NumPy uses **central differences**, which means:

* For the derivative in `x`, at point `(i, j)`:

  $$
  \frac{\partial f}{\partial x} \approx \frac{f(i, j+1) - f(i, j-1)}{2 \cdot \Delta x}
  $$

* For the derivative in `y`, at point `(i, j)`:

  $$
  \frac{\partial f}{\partial y} \approx \frac{f(i+1, j) - f(i-1, j)}{2 \cdot \Delta y}
  $$

So, each derivative uses only **two neighbors**:

* Left and right for `x` (columns before and after)
* Up and down for `y` (rows above and below)

**Not all 8 neighbors** are used — just the **direct neighbors along x and y**.

---
\+ smooth movements

\- is it biologically realistic?

could write code differently to use all 8 neigbours (see GPT)

__*Still need to check this code:*__ 
Could use a variation to compare results if there is time.

In [ ]:
import numpy as np

def gradient_8_neighbors(field):
    """
    Returns a direction vector (dx, dy) at each point in the field based on steepest descent using all 8 neighbors.
    """
    dx = np.zeros_like(field)
    dy = np.zeros_like(field)

    # Padded version to handle edges
    padded = np.pad(field, 1, mode='edge')
    
    directions = [
        (-1, -1), (-1,  0), (-1, +1),
        ( 0, -1),          ( 0, +1),
        (+1, -1), (+1,  0), (+1, +1),
    ]

    for i in range(1, field.shape[0]+1):
        for j in range(1, field.shape[1]+1):
            center = padded[i, j]
            max_grad = 0
            best_dx = 0
            best_dy = 0
            
            for dy_offset, dx_offset in directions:
                neighbor = padded[i + dy_offset, j + dx_offset]
                diff = center - neighbor  # higher to lower = descent
                grad = diff / np.sqrt(dx_offset**2 + dy_offset**2)  # normalize diagonal distance
                
                if grad > max_grad:
                    max_grad = grad
                    best_dx = dx_offset
                    best_dy = dy_offset
            
            dx[i-1, j-1] = best_dx
            dy[i-1, j-1] = best_dy

    return dx, dy


## Discussion points:

### Isotropy vs antisotropy

Our model uses __isotropic gradients__ - diffusion of repellent or is modeled as symetrical in x and y axis. This would be the case in a uniform medium, however tissues are not so uniform. Tissue fibers or cell bodies are influencing the diffusion in real biological systems. __Antisotropic__ gradients might better reflect this. 

---

### Axon navigation

#### 1. Compute axon trajectories using __np.gradient()__

* Currently using np.gradient() - calculating partial derivatives, uses central differences. 
* __NB:__ Computes the gradient of the __ENTIRE FIELD__ (your whole 2D array of potential values) __BEFORE__ the neuron starts growing.
* np.gradient() gives a smooth, continuous vector pointing in the steepest downhill direction based on real-valued differences.
* So np.gradient() lets your axons flow smoothly and follow curving trajectories—not just make stepwise hops.
* Good if you __ASSUME__ cues don't change significantly during the short time window of axon outgrowth.

\+ Our model is simplified and doesn't include fluctuations in gradients over time, so this would work and gives smooth paths, potentially faster

\- not suitable for dynamic changes - e.g. if you want to model changes in released molecules over time, e.g. reuptake of some chemical molecules by cells (biologically relevant)

##### Potential solutions:
* You could re-compute gradients dynamically if cues were changing.
* Or compute the gradient on-the-fly using only local sampling at each neuron’s position (e.g., using a 3×3 patch).

#### 2. Computing axon trajectories by comparing the 8 neighbours and chosing the lowest value to move toward

* That method is discrete and local—it only checks immediate neighbors.
* Potentially slower? 

Why not just “go toward the smaller number”?
That’s a simpler rule, and it’s similar to the "8-neighbor steepest descent" method we discussed.

---
### Noise
#### Math explainer part if included

2. Modeling Biological Noise 

To simulate the stochastic nature of molecular gradients and sensing noise, we add **Gaussian white noise**:

$$
F_{\text{noisy}}(x, y) = F(x, y) + \eta(x, y), \quad \eta(x, y) \sim \mathcal{N}(0, \sigma^2)
$$

The noise term $\eta(x, y)$ is sampled independently at each grid point. The amplitude $\sigma$ controls the noise intensity.

---

#### Total field noise
|Gausian white noise                    |Perlin noise|
|---                                    |---            |
|Easy to implement                      |X              |
|Jerky - steep differences              |Smooth - might more accurately represent tissues|
|Random - less danger of bias           |Without biological data to support parameters it <br> might bias the model wrongly|
|Mimics highly localized, rapidly changing <br> fluctuations like:<br> * receptor binding noise<br> * protein expression variability <br>     |Tissue-level inhomogeneities| 

Maybe we can do both?

#### Wandering neuron noise?

---

### Final field - Field superposition

Model uses linear superposition - it is a (linear) sum of the 2 repulsive fields and the attractive field

\+ Simple and easy to navigate

\- In biological systems some signals might have limitations in their actions e.g.:
* Axons have limited number of receptors that might be saturated and stronger signal wouldn't elicit stronger response.
* Some molecules might interact with each other to amplify / diminish response (like cathalizors for chemical reactions)

These effects are not accounted for when simply summing the fields.

---
### Limitations

#### Doesn't include axon-axon interaction

Axons themselves secrete Netrins which attract axons to each other helping them to form tracts (formations like cables). This could be addressed by introducing a __dynamic attractor field__ that strengthens along paths where multiple axons travel, mimicking this self-reinforcement mechanism.

#### Axons move though simulated cell bodies 

Axons currently pass through simulated cell bodies (lower part of the field). In reality, physical cell exclusion and contact-mediated guidance play important roles. This could be addressed by adding a masking field that represent impenetrable areas, improving spatial realism.




### Mathematical Model of Axon Navigation via Scalar Potential Fields and Gradients

Axon guidance during neural development is driven by chemical cues in the extracellular environment. These cues can be modeled mathematically as **scalar potential fields**: regions in space where concentrations of molecular attractants or repellents vary continuously.

#### Scalar Field Representation

We define a **total potential field** \( \phi(x, y) \in \mathbb{R} \) on a 2D spatial domain \([0,1] \times [0,1]\), representing the net effect of attractors (negative potential wells) and repellents (positive barriers). This is computed as a weighted sum of Gaussian sources:

\[
\phi(x, y) = \sum_{i=1}^{N_\text{amacrine}} A_i e^{-\frac{(x - x_i)^2 + (y - y_i)^2}{2\sigma^2_\text{amacrine}}}
+ \sum_{j=1}^{N_\text{muller}} M_j e^{-\frac{(x - x_j)^2 + (y - y_j)^2}{2\sigma^2_\text{muller}}}
- \sum_{k=1}^{N_\text{attractors}} T_k e^{-\frac{(x - x_k)^2 + (y - y_k)^2}{2\sigma^2_\text{attractor}}}
\]

Where:
- \( (x_i, y_i) \), \( (x_j, y_j) \), and \( (x_k, y_k) \) are spatial coordinates of **amacrine cells**, **Müller glia**, and **attractors** (e.g. optic disc).
- \( A_i, M_j, T_k \) are the strength parameters of each cue type.
- \( \sigma \) controls the spatial spread (standard deviation) of each field source.

This potential field \( \phi(x, y) \) is stored in `total_field` in the simulation code.

#### Gradient-Guided Movement

Neurons detect spatial differences in cue concentrations and extend their axons in the direction of **steepest descent** of the potential field. Mathematically, this corresponds to following the **negative gradient vector**:

\[
\vec{v}(x, y) = -\nabla \phi(x, y) = -\left( \frac{\partial \phi}{\partial x}, \frac{\partial \phi}{\partial y} \right)
\]

The gradient field \( \nabla \phi \) is computed using `np.gradient` in the simulation code. Each axon's position \( \vec{r}(t) = (x(t), y(t)) \) is updated iteratively using:

\[
\vec{r}(t+1) = \vec{r}(t) + \Delta t \cdot \vec{v}(x(t), y(t))
\]

This corresponds to a first-order **Euler integration** of the continuous gradient descent dynamics.

#### Discretization and Implementation

In code:
- The scalar field \( \phi(x, y) \) is discretized on a 2D grid (`grid_width` × `grid_height`).
- Gradients are approximated using finite differences via `np.gradient`.
- The neuron’s path is computed step-by-step by sampling the gradient at the nearest grid point and moving along the normalized negative gradient vector.
- If the gradient is zero (i.e., flat region), a random direction is chosen to mimic stochastic cell behavior.

#### Sources of Error and Biological Simplifications

1. **Discrete approximation**:  
   - The gradient is calculated on a grid; thus, resolution affects accuracy.
   - Euler integration can introduce errors, especially near steep gradients.

2. **Biological realism**:  
   - Real axons respond to multiple interacting guidance cues over time, not instantaneously.
   - Temporal changes and cell signaling cascades are not modeled here.

3. **Noise/randomness**:  
   - To simulate biological variability, randomness is introduced in starting positions and in movement when gradients are zero.

#### Visualization

- The **scalar potential** is visualized using a **heat map** (`imshow`) to represent concentration intensity.
- The **vector field** \( \nabla \phi \) can optionally be visualized using arrows (`quiver`) to illustrate direction and magnitude of axonal force.
- The **trajectories** of axons are shown as lines starting from initial positions and tracing their paths under gradient influence.

#### Summary

This model abstracts axon guidance as **gradient descent in a scalar field**, where chemical cues act as potential wells or barriers. While simplified, it captures key mathematical principles of navigation, optimization, and vector calculus, and provides a basis for simulating biologically inspired agent-based systems.
